In [12]:
import os
import torch
import cv2
import numpy as np
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms

# === Khởi tạo model FaceNet (đã pretrained trên VGGFace2) ===
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# === Tiền xử lý ảnh ===
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),  # [0,1]
    transforms.Normalize([0.5], [0.5])  # chuyển về [-1, 1]
])

# === Đọc ảnh crop sẵn từ thư mục ===
IMAGE_DIR = "facecrop"
embeddings = []
labels = []

for filename in os.listdir(IMAGE_DIR):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    path = os.path.join(IMAGE_DIR, filename)
    img_bgr = cv2.imread(path)
    if img_bgr is None:
        print(f"⚠️ Không đọc được ảnh {filename}")
        continue

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_tensor = transform(img_rgb).unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = model(img_tensor)
        embedding = embedding.cpu().numpy().flatten()

    label = filename.split("_")[0].lower()
    embeddings.append(embedding)
    labels.append(label)
    print(f"✅ Got embedding for {filename}")

print(f"\n✅ Extracted {len(embeddings)} embeddings")


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: operator torchvision::nms does not exist

In [ ]:
# === Huấn luyện SVM ===
if embeddings:
    X = np.array(embeddings)
    le = LabelEncoder()
    y = le.fit_transform(labels)

    clf = SVC(kernel="linear", probability=True)
    clf.fit(X, y)
   print("✅ SVM trained with", len(le.classes_), "classes")
else:
    print("❌ Không có embedding nào để train.")
